## EPCIS Document Signing Example

### Import libary

Make sure you have installed the epcis-signing package by executing `npm i epcis-signing`
If you are using a commonJS environment make sure to have installed `esm-hook` as well.

In [1]:
require('esm-hook');

const epcisSigning = require('epcis-signing');
const Ed25519VerificationKey2020 = require('@digitalbazaar/ed25519-verification-key-2020');

[Function (anonymous)]

### Create a dummy EPICS Document

For this example we will use a document containing an ObjectEvent

In [2]:
const epcisDocument = {
  "@context": [
    "https://ref.gs1.org/standards/epcis/2.0.0/epcis-context.jsonld"
  ],
  "id": "https://id.eecc.de/epcis/document/12345",
  "type": "EPCISDocument",
  "schemaVersion": "2.0",
  "creationDate": "2023-02-07T11:05:00.0Z",
  "epcisBody": {
    "eventList": [
      {
        "type": "ObjectEvent",
        "id": 'https://testid.com',
        "eventTime": "2023-02-07T11:04:03.15Z",
        "eventTimeZoneOffset": "+01:00",
        "epcList": [
          "https://id.eecc.de/01/04012345999990/21/XYZ-1234"
        ],
        "action": "OBSERVE",
        "bizStep": "repairing",
        "disposition": "conformant",
        "readPoint": {
          "id": "https://id.eecc.de/414/4012345000115"
        }
      }
    ]
  }
}

### Generate a key pair

In order to sign the event you will need a cryptographic key pair. We create an async function to do so.

In [3]:
async function getKeyPair() {

    // generate the keyPair from seed
    let keyPair = await Ed25519VerificationKey2020.Ed25519VerificationKey2020.generate();

    // name the keyPair in order to make it resolvable
    keyPair.id = 'did:key:' + keyPair.publicKeyMultibase + '#' + keyPair.publicKeyMultibase;
    keyPair.controller = 'did:key:' + keyPair.publicKeyMultibase;

    return keyPair;

}

### Create signing function

Create an ansyc signing function combining the signing and the key creation.,

In [4]:
async function signDocument(document) {

    const keyPair = await getKeyPair();
    
    console.log('--------------------------------Generated key pair-----------------------------------');
    
    console.log(keyPair);

    return await epcisSigning.sign(document, keyPair, 'https://test.com/vc/12345')

}

### Sign the document

Call the async `signDocument` function, wait for the promise to resolve and log the result.

In [5]:
signDocument(epcisDocument)
    .then((signedCred) => {
        console.log('----------------------Signed document as verifiable credential-----------------------');
        console.log(signedCred);
    });

Promise { <pending> }

--------------------------------Generated key pair-----------------------------------
Ed25519VerificationKey2020 {
  id: 'did:key:z6MkuweQW7Vbe9Za5YdPPNVVGU5CdH1wTM2FC4CCEZTHA9oj#z6MkuweQW7Vbe9Za5YdPPNVVGU5CdH1wTM2FC4CCEZTHA9oj',
  controller: 'did:key:z6MkuweQW7Vbe9Za5YdPPNVVGU5CdH1wTM2FC4CCEZTHA9oj',
  revoked: undefined,
  type: 'Ed25519VerificationKey2020',
  publicKeyMultibase: 'z6MkuweQW7Vbe9Za5YdPPNVVGU5CdH1wTM2FC4CCEZTHA9oj',
  privateKeyMultibase: 'zrv599ZwTHozbWabLFZJxU3Lvi3DaTpaoxvwJgrPhfU9xq8gjtr3VhARysfCuqXFLRQxVgccj2YfZ4yLH7m2zvENWpm'
}
------------------------Signed event as verifiable credential------------------------
{
  '@context': [
    'https://www.w3.org/2018/credentials/v1',
    'https://w3id.org/security/suites/ed25519-2020/v1'
  ],
  type: [ 'VerifiableCredential' ],
  id: 'https://test.com/vc/12345',
  issuer: 'did:key:z6MkuweQW7Vbe9Za5YdPPNVVGU5CdH1wTM2FC4CCEZTHA9oj',
  issuanceDate: '2023-05-09T08:48:02Z',
  credentialSubject: {
    '@context': [
      'http